In [9]:
import numpy as np 
import pandas as pd 
import os 
import pickle 
import warnings
warnings.filterwarnings("ignore")

## Remove the empty files from the KSC MERLIN Cloud to Cloud Dataset

In [2]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
for yr in yrs:
    for mo in mos:
        dir = data_dir+yr+'_CC/'+mo+'/'
        files = os.listdir(dir)
        for file in files:
            try:
                    temp = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False)
            except:
                print('no data: '+file+' removing')
                os.remove(dir+file)


no data: KSCCC20170127.dat removing
no data: KSCCC20170126.dat removing
no data: KSCCC20170130.dat removing
no data: KSCCC20170124.dat removing
no data: KSCCC20170118.dat removing
no data: KSCCC20170119.dat removing
no data: KSCCC20170125.dat removing
no data: KSCCC20170131.dat removing
no data: KSCCC20170109.dat removing
no data: KSCCC20170121.dat removing
no data: KSCCC20170120.dat removing
no data: KSCCC20170112.dat removing
no data: KSCCC20170113.dat removing
no data: KSCCC20170111.dat removing
no data: KSCCC20170110.dat removing
no data: KSCCC20170128.dat removing
no data: KSCCC20170114.dat removing
no data: KSCCC20170101.dat removing
no data: KSCCC20170115.dat removing
no data: KSCCC20170129.dat removing
no data: KSCCC20170103.dat removing
no data: KSCCC20170117.dat removing
no data: KSCCC20170116.dat removing
no data: KSCCC20170102.dat removing
no data: KSCCC20170226.dat removing
no data: KSCCC20170225.dat removing
no data: KSCCC20170224.dat removing
no data: KSCCC20170220.dat r

## Build a dataframe for each month

In [63]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
i=0
for yr in yrs:
    for mo in mos:
        print(mo)
        if mo=='06':
            first_file=True
            dir = data_dir+yr+'_CC/'+mo+'/'
            files = os.listdir(dir)
            for file in files:
                if first_file==True:
                    first_file=False
                    merlin_cc = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,20,22,24,26],low_memory=False,\
                        on_bad_lines='skip',parse_dates={"Date_Time":[0,2]})
                else:
                    print(file)
                    merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,20,22,24,26],\
                        on_bad_lines='skip',low_memory=False, parse_dates={"Date_Time":[0,2]}))
            data_stor = '/Users/brandonmcclung/Data/pickles/'
            file = open(data_stor+yr+'_'+mo+'_merlin_cc_df.p','wb')
            pickle.dump(merlin_cc,file)
            file.close()    
merlin_cc.set_index("Date_Time",inplace=True)
merlin_cc.rename({6:'Lat',8:'Lon',20:'SemiMajor',22:'SemiMinor',24:'Ellipse_Angle',26:'Sensors'},inplace=True,axis=1)
merlin_cc
#merlin_cc.columns=['Lat','Lon','SemiMajor','SemiMinor','EllipseAngle','Sensors']

01
02
03
04
05
06
KSCCC20170608.dat
KSCCC20170603.dat
KSCCC20170602.dat
KSCCC20170601.dat
KSCCC20170611.dat
KSCCC20170605.dat
KSCCC20170604.dat
KSCCC20170610.dat
KSCCC20170606.dat
KSCCC20170607.dat
07
08
09
10
11
12


,Lat,Lon,SemiMajor,SemiMinor,Ellipse_Angle,Sensors
Date_Time,,,,,,
2017-06-09 00:26:00.748000,27:45:00,-79:06:16,27.4,2.3,116.0,"62,63,64,68"
2017-06-09 04:02:29.010700,27:25:53,-79:08:33,25.4,2.4,125.0,"62,63,64,68"
2017-06-09 04:23:50.604700,27:27:58,-79:00:40,25.8,2.4,123.0,"62,64,65,66,68"
2017-06-09 21:42:16.458700,26:54:33,-80:52:25,17.8,1.5,1.0,"62,63,64,65,66,68"
2017-06-09 21:42:50.348700,26:47:01,-80:08:23,23.8,2.0,160.0,"62,63,64,65,66,68"
...,...,...,...,...,...,...
06/07/2017 22:18:45.5541000,26:43:56,-80:46:18,29.80,02.60,179,"62,64,65,66,68"
06/07/2017 22:18:46.3813000,26:59:52,-80:24:42,20.40,02.20,168,"62,65,66,68"
06/07/2017 22:22:49.2028000,26:56:43,-80:43:10,22.00,02.30,177,"62,64,66,68"


In [68]:
print(type(merlin_cc['Lat'][0]))

<class 'str'>
